In [3]:
import pandas as pd
import json

In [8]:
# loading the datasets
books = pd.read_json('./data/book_test.jl', lines=True)
interactives = pd.read_json('./data/interactive_test.json')

# Data at first sight

In [9]:
books.head(3)


,url,title,titleComplete,description,imageUrl,genres,asin,isbn,isbn13,publisher,...,author,publishDate,characters,places,ratingHistogram,ratingsCount,reviewsCount,numPages,language,awards
0,https://www.goodreads.com/book/show/2767052-th...,The Hunger Games,"The Hunger Games (The Hunger Games, #1)","Could you survive on your own in the wild, wit...",https://images-na.ssl-images-amazon.com/images...,"[Romance, Adventure, Dystopia, Fantasy, Scienc...",0439023483,0439023483,9.780439e+12,Scholastic Press,...,[Suzanne Collins],1223967600000,"[Rue, Effie Trinket, Haymitch Abernathy, Caesa...","[Capitol, Panem, District 12, Panem, Panem]","[118499, 214967, 958016, 2516526, 4483011]",8291019,208945,374,English,"[{'name': 'Locus Award', 'awardedAt': 12307968..."
1,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,Harry Potter and the Deathly Hallows (Harry Po...,"Harry has been burdened with a dark, dangerous...",https://images-na.ssl-images-amazon.com/images...,"[Classics, Adventure, Fantasy, Childrens, Scie...",NaN,NaN,NaN,Arthur A. Levine Books,...,[J.K. Rowling],1185001200000,"[Sirius Black, Ginny Weasley, Remus Lupin, Cho...","[London, England, Hogwarts School of Witchcraf...","[29882, 42756, 202691, 706484, 2610889]",3592702,82737,759,English,"[{'name': 'Locus Award', 'awardedAt': 11991744..."
2,https://www.goodreads.com/book/show/77203.The_...,The Kite Runner,The Kite Runner,1970s Afghanistan: Twelve-year-old Amir is des...,https://images-na.ssl-images-amazon.com/images...,"[Historical Fiction, Classics, Contemporary, H...",NaN,NaN,NaN,Riverhead Books,...,[Khaled Hosseini],1083394800000,"[Hassan (The Kite Runner), Amir (The Kite Runn...","[Pakistan, Kabul, Fremont, California, Afghani...","[46329, 82073, 318094, 952094, 1654764]",3053354,94878,371,English,"[{'name': 'Borders Original Voices Award', 'aw..."


In [10]:
books.columns

Index(['url', 'title', 'titleComplete', 'description', 'imageUrl', 'genres',
       'asin', 'isbn', 'isbn13', 'publisher', 'series', 'author',
       'publishDate', 'characters', 'places', 'ratingHistogram',
       'ratingsCount', 'reviewsCount', 'numPages', 'language', 'awards'],
      dtype='object')

In [11]:
# Drop columns
books.drop(['titleComplete', 'imageUrl', 'asin', 'isbn', 'isbn13', 'publisher', 'series', 'characters', 'places', 'awards'], axis=1, inplace = True)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              500 non-null    object
 1   title            500 non-null    object
 2   description      500 non-null    object
 3   genres           500 non-null    object
 4   author           500 non-null    object
 5   publishDate      500 non-null    int64 
 6   ratingHistogram  500 non-null    object
 7   ratingsCount     500 non-null    int64 
 8   reviewsCount     500 non-null    int64 
 9   numPages         500 non-null    int64 
 10  language         500 non-null    object
dtypes: int64(4), object(7)
memory usage: 43.1+ KB


In [12]:
interactives.head(3)

,book_url,title,user,user_rate,user_review
0,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,992038-tara,Rating 1 out of 5,"Ok, before I start a few warnings. This will c..."
1,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,167451-suzanne,Rating 5 out of 5,"""I'm going to keep going until I succeed — or ..."
2,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,8937622-kassidy,Rating 5 out of 5,It's hard for me to believe that I finished th...


In [13]:
interactives.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   book_url     1470 non-null   object
 1   title        1470 non-null   object
 2   user         1430 non-null   object
 3   user_rate    1434 non-null   object
 4   user_review  1470 non-null   object
dtypes: object(5)
memory usage: 57.6+ KB
